# Painel Fotovoltaico
### Autora: Luana Dias Costa Lucas <br> Empresa : WP SOLUÇÕES INDUSTRIAIS INTELIGENTES LTDA


## CLASSE HELPER

&emsp;Classe de apoio, contem métodos utilizados frequentemente pelas outras classes

&emsp;Referencias:

<ul>
<li>API previsão tempo - https://openweathermap.org/ </li>
<li>Extensão usada para buscar histórico do clima - https://dev.meteostat.net/python </li>
<li>Extensão usada para manusear os dados - https://pandas.pydata.org/docs/index.html </li> 
<li>Extensão para geração de pdfs - https://pyfpdf.readthedocs.io/en/latest/reference/set_y/index.html</li>
<li>Código usado como base para a geração dos relatórios - https://www.justintodata.com/generate-reports-with-python/</li>

In [7]:
#Bibliotecas
from datetime import date, datetime
from meteostat import Point, Daily
import sys
import numpy as np
import keyboard
import matplotlib.pyplot as plt
import pandas as pd
import requests

In [ ]:
def inputText(text):
    if sys.version_info.major == 2:
        reportNumber = raw_input(text)
    elif sys.version_info.major == 3:
        reportNumber = input(text)
    return reportNumber

In [21]:
# Pandas dataframe to PDF - spreadsheet
def spreadsheet_df_to_pdf(pdf, df, nColumns, margins, index):
    # Total width = 210, with 2 margins of 40 (total of 80), we can have 2 coluns with 65 of width
    if(index):
        table_cell_width = (210-2*margins-20)/nColumns
    else:
        table_cell_width = (210-2*margins)/nColumns
    table_cell_height = 6
    # Loop over to print each data in the table
    x = len(df.index)
    if x < 37 :
        cols = header(table_cell_width,table_cell_height, df, pdf, margins)
        for row in df.itertuples():
            pdf.cell(margins)
            columns(cols, row, table_cell_height, table_cell_width, pdf)
            pdf.ln(table_cell_height)
    else:
        i = np.ceil(x / 36)
        for y in range(int(i)):
            df_new = df.head(36)
            cols = header(table_cell_width, table_cell_height, df_new, pdf, margins)
            for row in df_new.itertuples():
                pdf.cell(margins)
                columns(cols, row, table_cell_height, table_cell_width, pdf)
                pdf.ln(table_cell_height)
            df = df.tail(len(df.index)-36)
            if y < i - 1:
                pdf.add_page()
                pdf.ln(10)

def header(table_cell_width, table_cell_height, df, pdf, margins):
    # Table header font: Arial, bold, 10
    pdf.set_font('Arial', 'B', 10)
    # Left margin 4cm for table
    pdf.cell(margins)
    # Loop over to print column names
    cols = df.columns
    for col in cols:
        if(col == 'index'):
            pdf.cell(20, table_cell_height, col, align='C', border=1)
        else:
            pdf.cell(table_cell_width, table_cell_height, col, align='C', border=1)
    # Line break
    pdf.ln(table_cell_height)
    # Table cells font: Arial, regular, 10
    pdf.set_font('Arial', '', 8)
    return cols

def columns(cols, row, table_cell_height, table_cell_width, pdf):
    for col in cols:
        value = str(getattr(row, col))
        if(col == 'index'):
            pdf.cell(20, table_cell_height, value, align='C', border=1)
        else:
            pdf.cell(table_cell_width, table_cell_height, value, align='C', border=1)

In [ ]:
# Pandas dataframe to PDF - inline
def inline_df_to_pdf(pdf, df):
    table_cell_height = 6
    margins = 20

    for row in df.itertuples():
        cols = df.columns
        for col in cols:
            pdf.cell(margins)
            pdf.set_font('Arial', 'B', 10)
            c = col + ': '
            pdf.cell(pdf.get_string_width(c), table_cell_height, c, align='L')
            value = str(getattr(row, col))
            pdf.set_font('Arial', '', 8)
            pdf.multi_cell(0, table_cell_height, value, align='L')
            pdf.ln(table_cell_height)   
        pdf.ln(table_cell_height)

In [ ]:
# Buscar ID paineis
def id_search(text, helpText, df):
    idNaoExiste = True
    ficarLoop = True
    source_index = 0
    while idNaoExiste and ficarLoop:
        sourceKey = inputText(text)
        try:
            if sourceKey in df['SOURCE_KEY'].values:
                idNaoExiste = False
                source_index = df[df['SOURCE_KEY']==sourceKey].index.to_numpy()
            elif keyboard.is_pressed('esc'):
                ficarLoop = False
            else:
                raise ValueError()
        except ValueError:
            print(helpText)
    return source_index

In [ ]:
# Inserir data formato AAAA-MM-DD
def insert_date(text, helpText):
    format = '%Y-%m-%d'
    wrongDate = True
    ficarLoop = True
    while wrongDate and ficarLoop:
        data = inputText(text)
        try:
            if data != '':
                if bool(datetime.strptime(data, format)):
                    wrongDate = False
            elif keyboard.is_pressed('esc'):
                ficarLoop = False
            else:
                raise ValueError()
        except ValueError:
            print(helpText)

    if not ficarLoop:
        data = date.today().strftime('%Y-%m-%d')
    
    return data

In [ ]:
# Busca os dados referente ao clima para certa localização, range de data e retorna a soma total da coluna (temp, chuva, etc)
def fetchWeatherData(start, end, lat, lon, column):
    # Pegar a localização
    location = Point(lat, lon)
    # Dados referente a esse periodo de tempo e localização
    data = Daily(location, start, end)
    data = data.fetch()
    # Soma a coluna selecionada e devolve a informação
    return data[column].sum()

In [9]:
# Previsão para os proximos 5 dias
def forecast():
    # API para buscar os dados
    api_key = '1c077c2df9585b5578b81e10fb3a4326'
    # Id de Florianópolis 
    city_id = '6323121'
    api_call = 'https://api.openweathermap.org/data/2.5/forecast?lang=pt_br&units=metric&appid=' + api_key + '&id=' + city_id

    # Chamado Json
    json_data = requests.get(api_call).json()
    data = np.array([['data', 'temp', 'clouds']])

    for item in json_data['list']:
        data = np.r_[data,[np.array([item['dt_txt'], item['main']['temp'], item['clouds']['all']])]]
    num_rows, num_cols = data.shape
    df = pd.DataFrame(data[1:num_rows], columns=data[0])
    return df

In [ ]:
# Escolha da central solar
def whichPlant():
    print('''
    Por favor digite 1 ou 2 para escolher uma das 2 centrais solares
    ''')

    notNumber = True
    while notNumber:
        plant = inputText("Insira o número desejado: ")
        try:
            val = int(plant)
            if 1 <= val <=2:
                notNumber = False
            else:
                raise ValueError()
        except ValueError:
            print("Por favor inserir um número entre 1 e 2")   
    return plant